## Skeleton for an UNet

<b> This implementation showcases a simple Unet
    
    The data manipulation methods are not present in the current 

In [1]:
# from  run_unet import *
# from Unet import *

In [1]:
train_mode = True
width_in = 284
height_in = 284
width_out = 196
height_out = 196
PATH = './unet.pt'

batch_size = 3
epochs = 1
epoch_lapse = 50
threshold = 0.5
learning_rate = 0.01

In [4]:
x_train, y_train, x_val, y_val = get_dataset(width_in, height_in, width_out, height_out)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.39s/it]

(99, 1, 284, 284) (99, 1, 196, 196) (11, 1, 284, 284) (11, 1, 196, 196)


In [20]:
unet = UNet(in_channel=1,out_channel=2)
unet = unet.cuda()

In [ ]:
#     train(unet, batch_size, epochs, epoch_lapse, threshold, learning_rate, criterion, optimizer, x_train, y_train, x_val, y_val, width_out, height_out)


In [21]:
unet.load_state_dict(torch.load(PATH))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [62]:
img = x_train[0]
img = np.transpose(img, (1,2,0))
img = np.resize(img, (254,284))
# plt.imshow(img)
# plt.show()

img = y_train[0]
img = np.transpose(img, (1,2,0))
img = np.resize(img, (254,284))
# plt.imshow(img)
# plt.show()

In [5]:
import torch.utils.data.dataloader

dataloader = torch.utils.data.DataLoader(x_train, batch_size = 5, shuffle = False, num_workers = 2)

img = iter(dataloader).next()
img.shape
# img = img.resize(img.shape[-1], img.shape[-1])

# plt.imshow(img)
# plt.show()

torch.Size([5, 1, 284, 284])

## Unet implementation

In [36]:
import torch.nn as nn
class MyUnet(nn.Module):
    
    def down_block(self, in_channels, out_channels, kernel_size = 3):
        print("Down Block: {} : {}".format(in_channels, out_channels))
        block = torch.nn.Sequential(
                    torch.nn.Conv2d(in_channels =in_channels, out_channels = out_channels, kernel_size = kernel_size),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(out_channels),
                    
                    torch.nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size = kernel_size),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(out_channels)
        )
        
        return block
    
    def up_block(self, in_channels, mid_channels, out_channels, kernel_size= 3):
        print("Up Block: {} : {} : {}".format(in_channels, mid_channels, out_channels))

        block = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels = in_channels, out_channels = mid_channels, kernel_size = kernel_size),
                torch.nn.ReLU(),
                torch.nn.BatchNorm2d(mid_channels),
            
                torch.nn.Conv2d(in_channels = mid_channels, out_channels = mid_channels, kernel_size = kernel_size),
                torch.nn.ReLU(),
                torch.nn.BatchNorm2d(mid_channels),
                torch.nn.ConvTranspose2d(in_channels = mid_channels, out_channels = out_channels, 
                                        kernel_size = kernel_size , stride = 2, padding = 1, output_padding = 1 )
        )
        
        return block
    
    
    def final_block(self, in_channels, mid_channels, out_channels, kernel_size=3):
        print("Final Block: {} : {} : {}".format(in_channels, mid_channels, out_channels))

        block = torch.nn.Sequential(
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=mid_channels),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(mid_channels),
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channels, out_channels=mid_channels),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(mid_channels),
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channels, out_channels=out_channels, padding=1),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(out_channels),
                )
        return  block
    
    def __init__(self):
        super(MyUnet, self).__init__()
        
        # Network structure
        in_channels = 1
        out_channels = 2
        
        #Encode
        self.conv_encode_1 = self.down_block(1, 64)
        self.conv_maxpool_1 = torch.nn.MaxPool2d(kernel_size = 2)
        self.conv_encode_2 = self.down_block(64, 128)
        self.conv_maxpool_2 = torch.nn.MaxPool2d(kernel_size = 2)
        self.conv_encode_3 = self.down_block(128, 256)
        self.conv_maxpool_3 = torch.nn.MaxPool2d(kernel_size = 2)

        #Bottleneck
        
        self.bottleneck = torch.nn.Sequential(
                            torch.nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3),
                            torch.nn.ReLU(),
                            torch.nn.BatchNorm2d(512),
                            torch.nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3),
                            torch.nn.ReLU(),
                            torch.nn.BatchNorm2d(512),
                            torch.nn.ConvTranspose2d(in_channels = 512, out_channels = 256, 
                                                         kernel_size = 3, stride = 2, padding = 1 , output_padding = 1)
        ) 
        
        #Decode
        self.conv_decode3 = self.up_block(512, 256, 128)
        self.conv_decode2 = self.up_block(256, 128, 64)
        self.final_layer = self.final_block(128, 64, out_channels)
        
    def crop_and_concat(self, upsampled, bypass, crop=False):
        if crop:
            c = (bypass.size()[2] - upsampled.size()[2]) // 2
            bypass = F.pad(bypass, (-c, -c, -c, -c))
        print("Concat dimensions {} : {} ".format(upsampled.shape , bypass.shape) )
        return torch.cat((upsampled, bypass), 1)
    
    def forward(self, x):
        #encode
        
        encode_block_1 = self.conv_encode_1(x)
        encode_pool_1 = self.conv_maxpool_1(encode_block_1)
        
        encode_block_2 = self.conv_encode_2(encode_pool_1)
        encode_pool_2 = self.conv_maxpool_2(encode_block_2)
        
        encode_block_3 = self.conv_encode_3(encode_pool_2)
        encode_pool_3 = self.conv_encode_3(encode_block_3)
        
        #bottleneck
        
        bottleneck1 = self.bottleneck(encode_pool_3)
        
        #decode
        
        decode_block3 = self.crop_and_concat(bottleneck1, encode_block3, crop=True)
        cat_layer2 = self.conv_decode3(decode_block3)
        decode_block2 = self.crop_and_concat(cat_layer2, encode_block2, crop=True)
        cat_layer1 = self.conv_decode2(decode_block2)
        decode_block1 = self.crop_and_concat(cat_layer1, encode_block1, crop=True)
        final_layer = self.final_layer(decode_block1)
        return  final_layer

In [71]:
import torch.optim as optim
import torch.nn.functional as functional

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(unet.parameters(), lr = 0.01, momentum=0.99)

In [74]:
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    
use_cuda

True

In [42]:
myUnet = MyUnet()
# myUnet.bottleneck

Down Block: 1 : 64
Down Block: 64 : 128
Down Block: 128 : 256
Up Block: 512 : 256 : 128
Up Block: 256 : 128 : 64
Final Block: 128 : 64 : 2


In [41]:
a = torch.ones(5,5)
b = torch.zeros(5,5)

c = torch.cat((a,b), 1)
c

tensor([[1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]])